<a href="https://colab.research.google.com/github/Vinoth-24/NLP/blob/main/Haystack/Haystack_for_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-5h2n_hl9/farm-haystack_6edc13e9ba4c4ce0a384e176e7ae6cd3
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-5h2n_hl9/farm-haystack_6edc13e9ba4c4ce0a384e176e7ae6cd3
  Resolved https://github.com/deepset-ai/haystack.git to commit ae712fe6bf087c717f3e38e4e87d2347165fc12b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 

In [2]:
pip install tf-estimator-nightly==2.8.0.dev2021122109


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 9.6 MB/s eta 0:00:00


In [3]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


In [5]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [6]:
# Connect to Elasticsearch

from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

INFO - haystack.telemetry -  Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by calling disable_telemetry() or by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems on the documentation page. More information at https://haystack.deepset.ai/guides/telemetry


In [7]:
doc_dir = "data/tutorial1"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to `data/tutorial1`


True

In [10]:
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting data/tutorial1/40_Stannis_Baratheon.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/511_After_the_Thrones.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/485_Oathkeeper.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/453_Game_of_Thrones__2012_video_game_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/24_Game_of_Thrones__Season_1__soundtrack_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/131_Mhysa.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/55_Breaker_of_Chains.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/228_Souad_Faress.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/102_The_Princess_and_the_Queen.txt
INFO - haystack.utils.preprocessing -  Converting data/tutorial1/75_Blackwater__Game_of_Thrones_.txt
INFO - haystack.utils.preprocessing -  Converting data/tutori

In [11]:
# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

[<Document: {'content': "'''Stannis Baratheon''' is a fictional character in the ''A Song of Ice and Fire'' series of epic fantasy novels by American author George R. R. Martin, and its television adaptation ''Game of Thrones''. He is the second son of Steffon Baratheon, the lord of Storm's End, and his wife Lady Cassana Estermont, and brother to King Robert and Renly. He is the Lord of Dragonstone and a claimant to the Iron Throne after his elder brother's death. Stannis's goals are frequently impeded by his lack of manpower and resources, owing to his unpopularity with other noble houses. He must therefore rely on the counsel of the foreign priestess Melisandre and his right-hand man, lowborn smuggler Davos Seaworth, who later becomes his Hand of the King. Stannis often struggles to escape the shadow of his two more overtly charismatic brothers, particularly Robert.\nThough first mentioned in 1996's ''A Game of Thrones'', Stannis formally appeared in ''A Clash of Kings'' (1998), ''A 

In [12]:
from haystack.nodes import ElasticsearchRetriever

retriever = ElasticsearchRetriever(document_store=document_store)

In [13]:
# Hugging Face's model hub (https://huggingface.co/models)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0  
INFO - haystack.modeling.infer -  /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \ 


In [14]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [22]:
query = "What is Jon Snow's role?"
prediction = pipe.run(
    query=query, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.04 Batches/s]


In [23]:
print_answers(prediction, details="minimum")


Query: What is Jon Snow's role?
Answers:
[   {   'answer': 'lord of Winterfell',
        'context': ' foil for Eddard:\n'
                   'Robb has risen to take his father’s place, as a lord of '
                   'Winterfell and as a focal character in the show. We never '
                   'saw what kind of w'},
    {   'answer': 'milk brother',
        'context': "he talked to Beric's squire Edric Dayne, who claims to be "
                   "Jon Snow's milk brother.  Gendry is later impressed with "
                   "Thoros of Myr's description of the "},
    {   'answer': "Night's Watch",
        'context': "n, which is ultimately successful. Jon's death releases "
                   "him from his Night's Watch vows and he decides to gather "
                   'allies to march with him on Winterfel'},
    {   'answer': 'resurrection',
        'context': 'yline where it serves a clearer purpose."\n'
                   "With respect to Jon Snow's resurrection, Jeremy Egner